this code is about the classification of the chest CT images into one of the four cancerous conditions namely adenocarcinoma, large cell carcinoma, squamous cell carcinoma and normal.

In [1]:
!pip install tensorflow-addons
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB7
from google.colab import drive
drive.mount('/content/gdrive')
import scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 27.1 MB/s 
Mounted at /content/gdrive


In [2]:
path = './gdrive/MyDrive/10-617 Project Data'
img_size = 600

In [3]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(    
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)
test_datagen= image.ImageDataGenerator(    
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [4]:
train_generator = train_datagen.flow_from_directory(
    f'{path}/Data/train',
    target_size = (img_size,img_size),
    batch_size = 32,
    shuffle=True,
    class_mode = 'categorical')
    
validation_generator = val_datagen.flow_from_directory(
    f'{path}/Data/valid',
    target_size = (img_size,img_size),
    batch_size = 1,
    shuffle=True,
    class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(
    f'{path}/Data/test',
    target_size = (img_size,img_size),
    batch_size = 1,
    shuffle=True,
    class_mode = 'categorical')

Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
Found 315 images belonging to 4 classes.


I have developed a transfer learning based EfficientNetB3 model that has been trained on imagenet dataset. I have added some additional layers to the basic model.

In [5]:
#base_model = EfficientNetB7(weights='imagenet', classifier_activation="softmax", include_top=False)
base_model = EfficientNetB7(weights='imagenet', classifier_activation="softmax", include_top=False)

print(len(base_model.layers))
#print("Layers:", base_model.layers)
lr = 0.1
lr_mult = 0.9

parameters = []

i = 0
for layer in base_model.layers:
    layer.trainable=True

model = Sequential()
model.add(base_model)
model.add(GlobalMaxPooling2D(name="gap"))
model.add(Dense(4, activation='softmax'))
model.summary()


#model.compile(loss='categorical_crossentropy',
              #optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
              #metrics=['accuracy','AUC','Precision','Recall'])
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=False)
optimizers = [tf.keras.optimizers.Adam(learning_rate = lr_schedule), tf.keras.optimizers.Adam(learning_rate = 0.0001)]
optimizers_and_layers = [(optimizers[0], model.layers[0]), (optimizers[1], model.layers[1:])]

model.compile(loss='categorical_crossentropy',
              optimizer=tfa.optimizers.MultiOptimizer(optimizers_and_layers),
              metrics=['accuracy','AUC','Precision','Recall'])
              
#model.compile(loss='categorical_crossentropy',
              #optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule),
              #metrics=['accuracy','AUC','Precision','Recall'])

258076736/258076736 [==============================] - 12s 0us/step
814
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb7 (Functional)  (None, None, None, 2560)  64097687 
                                                                 
 gap (GlobalMaxPooling2D)    (None, 2560)              0         
                                                                 
 dense (Dense)               (None, 4)                 10244     
                                                                 
Total params: 64,107,931
Trainable params: 63,797,204
Non-trainable params: 310,727
_________________________________________________________________


In [8]:
base_model.summary()

Model: "efficientnetb7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling (Rescaling)          (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 normalization (Normalization)  (None, None, None,   7           ['rescaling[0][0]']              
                                3)                                                   

In [7]:
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
)

Epoch 1/30


ResourceExhaustedError: ignored

In [ ]:
print(history.history.keys())
h = history.history
print(h['val_accuracy'])

In [ ]:
hdict = history.history
keys = list(hdict.keys())
print(keys)
def graph(name, index):
    print(name, keys[index], keys[(len(keys) // 2) + index])
    train_metric, test_metric = hdict[keys[index]], hdict[keys[(len(keys) // 2) + index]]
    epochs = len(train_metric)
    x = list(range(epochs))

    plt.plot(x, train_metric, label=f'train {name}')
    plt.plot(x, test_metric, label=f'test {name}')

    plt.xlabel('Epoch #')
    plt.ylabel(name.capitalize())
    plt.legend(loc='lower left')
    plt.savefig(f'./initial_plots/{name}.png')
    plt.show()
    plt.clf()

graph('loss', 0)
graph('accuracy', 1)
graph('precision', 2)
graph('recall', 4)

In [ ]:
model.evaluate(train_generator)

In [ ]:
model.evaluate(validation_generator)


In [ ]:
model.evaluate(test_generator)
# loss, 'accuracy','AUC','Precision','Recall'

In [ ]:
# from keras.preprocessing import image
# img = image.load_img('/content/Data/valid/squamous_cell_carcinoma_left.lower.lobe_T2_N0_M0_Ib/000117 (6).png',target_size=(224,224))
# imag = image.img_to_array(img)
# imaga = np.expand_dims(imag,axis=0) 
# ypred = model.predict(imaga)
# print(ypred)
# a=np.argmax(ypred,-1)
# if a==0:
#   op="Adenocarcinoma"
# elif a==1:
#   op="large cell carcinoma"
# elif a==2:
#   op="normal (void of cancer)"
# else:
#   op="squamous cell carcinoma"    
# plt.imshow(img)
# print("THE UPLOADED IMAGE IS SUSPECTED AS: "+str(op))  


In [ ]:
tf.keras.models.save_model(model,'mymodel.hdf5')